# Fine-tuning EmbeddingGemma pour Pocket Arbiter

**ISO Reference**: ISO/IEC 42001 A.6.2.2, ISO/IEC 25010 S4.2

Ce notebook fine-tune EmbeddingGemma sur le domaine echecs/arbitrage FR.

**Instructions**: Clique sur **Exécution** → **Tout exécuter** (c'est tout!)

In [ ]:
# Installation des dependances
!pip install -q sentence-transformers datasets accelerate

In [ ]:
# Verification GPU
import torch
print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Telechargement automatique des donnees depuis GitHub
!wget -q https://raw.githubusercontent.com/pierrealexandreguillemin-a11y/pocket_arbiter/main/data/training/triplets_training.jsonl
print("Donnees telechargees!")

In [ ]:
# Chargement des triplets
import json

triplets = []
with open("triplets_training.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            triplets.append(json.loads(line))

print(f"Triplets charges: {len(triplets)}")
print(f"Exemple: {triplets[0]['anchor'][:50]}...")

In [ ]:
# Configuration
# Note: multilingual-e5-base ne necessite pas d'authentification HuggingFace
MODEL_ID = "intfloat/multilingual-e5-base"
OUTPUT_DIR = "embedding-chess-fr"
EPOCHS = 3
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
WARMUP_RATIO = 0.1

In [ ]:
# Chargement du modele
from sentence_transformers import SentenceTransformer

print(f"Chargement de {MODEL_ID}...")
model = SentenceTransformer(MODEL_ID)
print(f"Dimension: {model.get_sentence_embedding_dimension()}")

In [ ]:
# Preparation du dataset
from datasets import Dataset

dataset = Dataset.from_list(triplets)
print(f"Dataset: {len(dataset)} examples")
print(dataset)

In [ ]:
# Configuration du training
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss

# Loss function
loss = MultipleNegativesRankingLoss(model)

# Arguments
args = SentenceTransformerTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_ratio=WARMUP_RATIO,
    fp16=True,  # Mixed precision pour GPU
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

print("Training args configured")

In [ ]:
# Entrainement
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    loss=loss,
)

print("Debut de l'entrainement...")
trainer.train()
print("Entrainement termine!")

In [ ]:
# Sauvegarde du modele
model.save(OUTPUT_DIR)
print(f"Modele sauvegarde dans {OUTPUT_DIR}/")

In [ ]:
# Test du modele
test_queries = [
    "Quelle est la regle du toucher-jouer ?",
    "Comment fonctionne le roque ?",
    "Que faire en cas de partie nulle ?",
]

print("Test du modele fine-tune:")
for q in test_queries:
    emb = model.encode(q)
    print(f"  {q[:40]}... -> {emb.shape}")

In [ ]:
# Telechargement du modele
import shutil
from google.colab import files

# Creer une archive
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)
print(f"Archive creee: {OUTPUT_DIR}.zip")

# Telecharger
files.download(f"{OUTPUT_DIR}.zip")
print("Telechargement lance!")

## Prochaines etapes

1. Extraire `embeddinggemma-chess-fr.zip` dans `models/`
2. Executer `python -m scripts.training.evaluate_finetuned`
3. Si recall >= 80%, regenerer les embeddings du corpus